# Trip Advisor model

## Steps

1. Import Trip Advisor data
2. Tokenize the data (create a word index that represents words as numbers)
3. Use an oov token to include words not seen before
4. Pad the sentences to have similar length


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
sentences = list(df['Review'])
labels = list(df['Rating'])

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, random_state=0)

print(X_train[:5])
print(y_train[-5:])

['fabulous hotel mum just returned 4 night stay hotel 1898 fabulous, recently decorated rooms immaculate decor fantastic really beautiful hotel, location perfect nicer end la ramblas need close treat return tranquility hotel busy day sight seeing.the staff attentive polite spoke perfect english, price drinks bar expect little hard swallow having gorgeous course meal local resturant price gin tonics.although roof pool undergoing refurbishment stay basement pool surrounding facilities adequate,  ', "romantic international ambience spent honeymoon melia caribe 23-30. plane landed torrential downpour soaked skin steps plane, rained 7 days just hot gorgeous, truly loved resort food people, management helpful needed courteous friendly, nightly shows fun casino, pools incredible beach beautiful, just short stroll resort swim deserted stretches beach wanted, took outback tour must-do tourists, islanders live visit mountains macou beach enjoy lunch siesta hammocks, buy souviniers tour rum 2/bot

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Tokenize the words (bag of words) with an oov token
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [7]:
import numpy as np

In [8]:
sequences = tokenizer.texts_to_sequences(X_test)
padded = pad_sequences(sequences, padding='post', maxlen=int(np.median([len(x) for x in X_test])))

print(padded[0])
print(padded.shape)

[15893    65   218   216   432   767     2    16     2   436   107   239
   289   387     7    13    35  2864    67    13  4626  1187  1381  1148
 31253   129    12   244  3959   173  5356     8    28    12    41  1477
     3    10     4   538    25    33 15894   352    12    35   320    28
   391   167  4696  5018   434   103    57     8  1477    12     8   705
  6889   517    13     6   112   496  1182    11  1528   341     3   642
   194  1263     6 31254  1148 15557   806    21   512  3186    30    57
    20  1157     4    25    39    25  5110   935  2864   248  1208   258
   111   225     2    31    44    42   723    25  5870  1564    65     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   